In [1]:
import sparknlp
from pyspark.sql import SQLContext

from LogisticRegressionCached import readFromCache

In [2]:
spark = sparknlp.start()
sc = spark.sparkContext
sqlCtx = SQLContext(sc)

In [3]:
(lrModel, pipelineModel) = readFromCache('./train/')

def score_text(text):
    df = spark.createDataFrame([(text, 2)], ['text', 'target'])
    df_transformed = pipelineModel.transform(df) # To fix
    predictions = lrModel.transform(df_transformed)
    predictions = predictions.select(['text', 'probability', 'prediction'])
    pd_predictions = predictions.toPandas()
    positive_probability = pd_predictions.iloc[0]['probability'][1]
    overall_probability = 2 * positive_probability - 1
    
    return overall_probability

models loaded


In [5]:
score_text("good")

0.5232641500469681

In [6]:
score_text("bad")

-0.4497107663079577

In [7]:
paragraph1 = "Mathematics (from Greek μάθημα máthēma, knowledge, study, learning) includes the study of such topics as quantity,[1] structure,[2] space,[1] and change.[3][4][5] It has no generally accepted definition.[6][7]"
paragraph2 = "Mathematicians seek and use patterns[8][9] to formulate new conjectures; they resolve the truth or falsity of conjectures by mathematical proof. When mathematical structures are good models of real phenomena, then mathematical reasoning can provide insight or predictions about nature. Through the use of abstraction and logic, mathematics developed from counting, calculation, measurement, and the systematic study of the shapes and motions of physical objects. Practical mathematics has been a human activity from as far back as written records exist. The research required to solve mathematical problems can take years or even centuries of sustained inquiry."
paragraph3 = "Rigorous arguments first appeared in Greek mathematics, most notably in Euclid's Elements. Since the pioneering work of Giuseppe Peano (1858–1932), David Hilbert (1862–1943), and others on axiomatic systems in the late 19th century, it has become customary to view mathematical research as establishing truth by rigorous deduction from appropriately chosen axioms and definitions. Mathematics developed at a relatively slow pace until the Renaissance, when mathematical innovations interacting with new scientific discoveries led to a rapid increase in the rate of mathematical discovery that has continued to the present day.[10]"

print(len(paragraph1), len(paragraph2), len(paragraph3))

209 661 630


In [8]:
score_text(paragraph1)

0.9572118814863957

In [9]:
print(score_text(paragraph2))
print(score_text(paragraph3))
print(score_text(paragraph1 + paragraph3))
print(score_text(paragraph2 + paragraph3))

0.9212783119197854
-0.9814452916592395
-0.757282531786352
-0.8891352743936631


In [ ]:
spark.stop()